[colab](https://colab.research.google.com/github/rufous86/studies/blob/main/DLS_NLP/02_hw_embedding_based_word_ranking/solution_simple_embeddings.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/Study/courses/DLS - NLP/2. [HW] Ранжирование слов на основе эмбедингов/stackoverflow_similar_questions.zip' .
!unzip stackoverflow_similar_questions.zip

Mounted at /content/drive
Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [2]:
# !wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [3]:
# from gensim.models.keyedvectors import KeyedVectors
# wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

In [4]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Study/courses/DLS - NLP/2. [HW] Ранжирование слов на основе эмбедингов/SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [5]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [6]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [7]:
# method most_simmilar
'''your code'''
import numpy as np
import pandas as pd

dog_embedding = wv_embeddings['dog']
cosine_with_dog = {}
for word in wv_embeddings.index_to_key:
    cosine = np.dot(dog_embedding, wv_embeddings[word]).T/(np.linalg.norm(dog_embedding)*np.linalg.norm(wv_embeddings[word]))
    cosine_with_dog[word] = cosine

cosine_with_dog_df = pd.DataFrame(cosine_with_dog, index=['cosine']).T.sort_values(by='cosine', ascending=False).reset_index()
cosine_with_dog_df.iloc[1:6, :]

,index,cosine
1,animal,0.856418
2,dogs,0.788087
3,mammal,0.762380
4,cats,0.762125
5,animals,0.760794


In [8]:
cosine_with_dog_df.loc[cosine_with_dog_df['index'] == 'cat', :]

,index,cosine
26,cat,0.685234


Слово слов `cat` не входит в топ-5 близких слов к слову `dog`. Оно находится на 26-м месте

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [9]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
# class MyTokenizer:
#     def __init__(self):
#         pass
#     def tokenize(self, text):
#         return re.findall('\w+', text)
# tokenizer = MyTokenizer()

In [10]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''
    tokens = tokenizer.tokenize(question.lower())
    words_in_question = []
    for token in tokens:
        if token in embeddings:
            words_in_question.append(embeddings[token])
    if not words_in_question:
        return np.zeros((dim,))
    return np.mean(words_in_question, axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [11]:
'''your code'''
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

question_to_vec('I love neural networks', 
                wv_embeddings,
                tokenizer).round(2)[2]

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ: Максимум (`Hits@47 - DCG@1`) = максимум `Hits@47` - минимум `DCG@1` = 1 - 0 = 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [12]:
import math

round(1 / math.log2(9+1), 1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [13]:
from math import log2

def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    hits_value = [int(dup_rank<=k) for dup_rank in dup_ranks]
    return sum(hits_value) / len(hits_value)

In [14]:
def dcg_score(dup_ranks, k): 
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    
    dcg_value = [(1 / log2(dup_rank+1)*int(dup_rank<=k)) for dup_rank in dup_ranks]
    return sum(dcg_value) / len(dcg_value)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [15]:
import pandas as pd
import numpy as np

In [16]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [17]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

Считайте данные.

In [18]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [19]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [20]:
len(validation_data)

3760

Размер нескольких первых строк

In [21]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [23]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    q_vec = question_to_vec(question, embeddings, tokenizer, dim)
    cand_vecs = []
    for candidate in candidates:
        cand_vecs.append(question_to_vec(candidate, embeddings, tokenizer, dim))

    cand_dict = {}
    for i, cand_vec in enumerate(cand_vecs):
        cand_dict[(i, candidates[i])] = cosine_similarity(q_vec.reshape(1, -1), cand_vec.reshape(1, -1))[0][0]
        result = np.array((sorted(cand_dict.items(), key=lambda item: item[1], reverse=True)))[:,0]
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [24]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [25]:
tokenizer = WordPunctTokenizer()
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it')
 (0, 'Convert Google results object (pure js) to Python object')
 (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP')
 (2, 'select2 not displaying search results')
 (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [26]:
# # должно вывести
# results = [[(1, 'C# create cookie from string and send it'),
#             (0, 'Convert Google results object (pure js) to Python object'),
#             (2, 'How to use jQuery AJAX for an outside domain?')],
#            [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
#             (*, 'select2 not displaying search results'), #скрыт
#             (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: `021`

### Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [27]:
from tqdm.notebook import tqdm

In [28]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [29]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.410 | Hits@   1: 0.410
DCG@   5: 0.500 | Hits@   5: 0.580
DCG@  10: 0.521 | Hits@  10: 0.645
DCG@ 100: 0.568 | Hits@ 100: 0.875
DCG@ 500: 0.581 | Hits@ 500: 0.973
DCG@1000: 0.583 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [30]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [31]:
'''your code'''
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, TreebankWordTokenizer

stops = set(stopwords.words('english'))
punkts = set()

train_data_tokenized = []
tokenizer = TreebankWordTokenizer()

for data in tqdm(train_data):
    tokens = [word for word in 
                        tokenizer.tokenize(' '.join(data).lower()) 
                                        if word not in stops]
    train_data_tokenized.append(tokens)
print('len_train_data', len(train_data), 'len_train_data_tokenized', len(train_data_tokenized))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  0%|          | 0/1000000 [00:00<?, ?it/s]

len_train_data 1000000 len_train_data_tokenized 1000000


In [67]:
import gensim
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(train_data_tokenized, # data for model to train on
                vector_size=200,    # embedding vector size
                min_count=5,        # consider words that occured at least 5 times '''your code'''
                window=5).wv        #'''your code'''

In [68]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [69]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.385 | Hits@   1: 0.385
DCG@   5: 0.472 | Hits@   5: 0.547
DCG@  10: 0.497 | Hits@  10: 0.625
DCG@ 100: 0.542 | Hits@ 100: 0.847
DCG@ 500: 0.559 | Hits@ 500: 0.971
DCG@1000: 0.562 | Hits@1000: 1.000


### Попробуем нормализовать векторы

In [70]:
from sklearn.preprocessing import normalize

normalized_vectors = normalize(embeddings_trained.vectors)
embeddings_trained_normed = gensim.models.KeyedVectors(vector_size=embeddings_trained.vector_size)
embeddings_trained_normed.add_vectors(embeddings_trained.index_to_key, normalized_vectors)

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained_normed, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


- Лучшее качество дала токенезация